In [1]:
# Imports

import os
import math
import random
import operator
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import math, itertools
import statistics

from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import ShuffleSplit
from operator import itemgetter
from statistics import mean
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from collections import Counter
# from ipynb.fs.full.helper_fns import *

In [5]:
# Helper Functions

def load_file(data_file_path):
    data_df = pd.read_csv(data_file_path, delimiter=";")
    return data_df
  
    
def plot_graph_new(results, max_calls, list_passed, title):
    x_pts = [i+1 for i in range(0, max_calls)]
    if list_passed:
        y_pts = results
    else:    
        y_pts = [results[i]['expected'] for i in range(0, max_calls)]
    plt.title(title)
    plt.plot(x_pts, y_pts)
    plt.axvline(x=0, color ="black")
    plt.axhline(y=0, color ="black")
    plt.xticks(np.arange(1, max_calls+1, 1))
    plt.show()
    

# def plot_graph_both_axes(ratios, no_calls):
    

def div(a,b):
    if int(b) == 0:
        return 0.0
    else:
        return a/b
    

# Used for creating all possible combinations of the features.
def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = iterable
    return itertools.chain.from_iterable(combinations(s, r) for r in range(len(s)+1))


def convert(list): 
    return tuple(list) 


def construct_dict(feature_comb):
    new_dict = {}
    new_dict['education'] = convert(feature_comb[0])
    new_dict['job'] = convert(feature_comb[2])
    new_dict['marital'] = convert(feature_comb[1])
    new_dict['default'] = convert(feature_comb[3])
    new_dict['loan'] = convert(feature_comb[4])
    new_dict['housing'] = convert(feature_comb[5])
    return new_dict


# This was the old metric (reward per call rate).
# def compute_expected_reward_feature_set_new(fs_df, no_calls_considered):
#     expected_values_calls = []
#     len_df = len(fs_df)
#     for i in range(1, no_calls_considered + 1):
#         expected_values_calls.append({'neg_value':0.0, 'pos_value':0.0, 'count':0, 'expected':0.0})
#         for index, row in fs_df.iterrows():
#             no_calls = row['campaign']
#             if no_calls <= i:
#                 if row['y'] == "yes":
#                     expected_values_calls[i-1]['pos_value'] += ((no_calls_considered+1) - no_calls)
#                 else:
#                     expected_values_calls[i-1]['neg_value'] += (-no_calls)
#             else:
#                 expected_values_calls[i-1]['neg_value'] += (-i)
#             expected_values_calls[i-1]['count'] += 1
#     for loc, item in enumerate(expected_values_calls):
#         expected_values_calls[loc]['expected'] = (expected_values_calls[loc]['pos_value'] + expected_values_calls[loc]['neg_value'])/len_df
#     return expected_values_calls


# This is the new metric (success per call rate).
def compute_expected_succ_per_call_rate_feature_set(fs_df, no_calls_considered):
    expected_values_call_nums = []
    for i in range(1, no_calls_considered + 1):
        expected_values_call_nums.append({'succ':0, 'total_calls':0, 'expected':0.0})
        for index, row in fs_df.iterrows():
            no_calls = row['campaign']
            if no_calls <= i:
                if row['y'] == "yes":
                    expected_values_call_nums[i-1]['succ'] += 1
                expected_values_call_nums[i-1]['total_calls'] += no_calls
            else:
                expected_values_call_nums[i-1]['total_calls'] += i
    for loc, item in enumerate(expected_values_call_nums):
        expected_values_call_nums[loc]['expected'] = div(item['succ'], item['total_calls'])
    return expected_values_call_nums


def compute_optimal_call_no(results):
    max_loc = max(range(len(results)), key=lambda index: results[index]['expected'])
    if max_loc == 0 and results[max_loc]['expected'] == 0.0:
        return -1
    return max_loc


# Given a dictionary of what attributes comprise a feature set, we can get all rows corresponding to this feature set.
def extract_rows_feature_set(fs_df, feature_labels = {'education':['tertiary', 'unknown'], 
                                                      'job':['management', 'technician', 'blue-collar'], 
                                                      'marital':['single'], 'default':['no'], 
                                                      'housing':['no'], 'loan':['no']}):
    for key in feature_labels:
        feature_labels_query_str = ''
        arr = feature_labels[key]
        for label in arr:
            feature_labels_query_str += (key + ' == "'+ label + '" | ')
        feature_labels_query_str = feature_labels_query_str[:-3]
        fs_df = fs_df.query(feature_labels_query_str)
    return fs_df


def find_matching_attribute_comb(row_value, all_combs):
    query = None
    for comb in all_combs:
        for item in comb:
            if item == row_value:
                query = comb
    return query


def compute_metric(df):
    total_calls = 0
    total_successes = 0
    for loc, row in df.iterrows():
        if row['y'] == "yes":
            total_successes += 1
        total_calls += row['campaign']
    return div(total_successes, total_calls)

def compute_metric_2(df):
    total_calls = 0
    total_successes = 0
    for loc, row in df.iterrows():
        if row['y'] == "yes":
            total_successes += 1
        total_calls += min(row['campaign'], )
    return div(total_successes, total_calls)


def compute_metric_for_each_attribute(all_values, df, attrib):
    metric_vals = np.zeros(shape=(len(all_values),1))
    for index, value in enumerate(all_values):
        v_query = "{0} == '{1}'".format(attrib, value)
        dataset_query = df.query(v_query)
        metric_val = compute_metric(dataset_query)
        metric_vals[index] = metric_val
#         print(v_query, metric_val, dataset_query.shape)
    return metric_vals


def compute_metric_for_each_attribute_range(all_values, df, attrib):
    metric_vals = np.zeros(shape=(len(all_values),1))
    query_strings = []
    for index, value in enumerate(all_values):
        v_query = "{0} >= {1} & {2} < {3}".format(attrib, value[0], attrib, value[1])
        dataset_query = df.query(v_query)
        metric_val = compute_metric(dataset_query)
        metric_vals[index] = metric_val
        query_strings.append(v_query)
#         print(v_query, metric_val, dataset_query.shape)
    return metric_vals, query_strings


def find_combinations(sub_attributes, ratios):
    num_iter = len(ratios)
    sil_scores = []
    # Making use of the K-Means algorithm ... number of centroids are from 2 to n-1.
    for clust_num in range(2, num_iter):
        kmeans = KMeans(n_clusters = clust_num)
        kmeans.fit(ratios.reshape(-1,1))
        results = kmeans.labels_
        sil_scores.append((silhouette_score(ratios.reshape(-1,1), results, metric='euclidean'), results, clust_num))
    # We make use of the silhouette score to determine the ideal number of centroids.
    sorted_sil_scores = sorted(sil_scores, key=lambda x: x[0], reverse = True)
    # We then use this ideal number of centroids to determine which sub attributes should be aggregated.
    joined_sub_attributes = []
    for i in range(0, sorted_sil_scores[0][2]):
        joined_sub_attributes.append([])
    join_list = sorted_sil_scores[0][1]
    for index, value in enumerate(join_list):
        pos = join_list[index]
        joined_sub_attributes[pos].append(sub_attributes[index])
    return_joined_sub_attributes = []
    for arr in joined_sub_attributes:
        similar_els_gp = []
        for item in arr:
            similar_els_gp.append(str(item))
        return_joined_sub_attributes.append(similar_els_gp)
#     print(return_joined_sub_attributes)
    return return_joined_sub_attributes

# The following is the format of the way in which this method should be called.
# find_combinations(['a', 'b', 'c', 'd'], np.array([1, 4, 7, 90]), "job").

In [6]:
%%time
# Code that sets up values to construct all possible feature combinations.

# Age query strings.
# age_query_strings = ['age < 26','age >= 26 & age <=60','age >60']
# age_query_strings = ['age >= 10 & age <= 32', 'age >= 33 & age <= 40', 'age >= 50 & age <= 59', 'age >= 60']
age_query_strings = ['age >= 10 & age <= 34', 'age >= 35 & age <= 45', 'age >= 46']

# Balance query strings.
balance_query_strings = ['balance <= 450',' balance > 450']

# Max call number to consider.
max_calls = 20

# Pull and filter all calls <= 20.
current_dir = os.getcwd()
mkt_df = load_file(current_dir + '/bank-full.csv')
mkt_df_filtered = mkt_df[(mkt_df['campaign']>=1) & (mkt_df['campaign']<=max_calls)]

print(mkt_df_filtered.shape)

(44967, 17)
CPU times: user 63.3 ms, sys: 19.2 ms, total: 82.6 ms
Wall time: 82.3 ms


In [7]:
%%time

# Main code ... orchestrates everything!

# Splitting dataframe into data and result dataframes.
X = mkt_df_filtered.iloc[:,0:len(mkt_df_filtered.columns)-1]
y = mkt_df_filtered.iloc[:,-1]   

all_call_limits = [500, 1000, 1500]

for call_limit in all_call_limits:
    all_non_optimal_ratios = []
    all_optimal_ratios = []
    for j in range(1,11):
        i = 0
        kf = KFold(n_splits=5, shuffle=True)

        for train_index, test_index in kf.split(X):
            i += 1

            train_df = mkt_df_filtered.iloc[train_index]
            test_df = mkt_df_filtered.iloc[test_index]

            # At this point, we can run computations for the success rate of each sub attribute and join
            # the sub-attributes based on the output of k-means.
            poss = []

            # Education.
            all_ed = ['tertiary', 'secondary', 'primary', 'unknown']
            metric_vals = compute_metric_for_each_attribute(all_ed, train_df, 'education')
            education_cmbs = find_combinations(all_ed, metric_vals)

            # Occupation.
            all_jobs = ['student', 'retired', 'unemployed', 'admin.', 'management', 'self-employed', 'technician', 'unknown', 'services', 'housemaid', 'blue-collar', 'entrepreneur']
            metric_vals = compute_metric_for_each_attribute(all_jobs, train_df, 'job')
            job_cmbs = find_combinations(all_jobs, metric_vals)

            # Marital.
            all_ms = ['married', 'single', 'divorced', 'unknown']
            metric_vals = compute_metric_for_each_attribute(all_ms, train_df, 'marital')
            marital_cmbs = find_combinations(all_ms, metric_vals)

            # Default
            all_def = ['no', 'yes', 'unknown']
            metric_vals = compute_metric_for_each_attribute(all_def, train_df, 'default')
            default_cmbs = find_combinations(all_def, metric_vals)

            # Loan
            all_ln = ['no', 'yes', 'unknown']
            metric_vals = compute_metric_for_each_attribute(all_ln, train_df, 'loan')
            loan_cmbs = find_combinations(all_ln, metric_vals)

            # Housing
            all_hs = ['no', 'yes', 'unknown']
            metric_vals = compute_metric_for_each_attribute(all_hs, train_df, 'housing')
            housing_cmbs = find_combinations(all_hs, metric_vals)

            poss.append(education_cmbs)
            poss.append(marital_cmbs)
            poss.append(job_cmbs)
            poss.append(default_cmbs)
            poss.append(loan_cmbs)
            poss.append(housing_cmbs)
            all_combs = list(itertools.product(*poss))

            # print("Number of combinations: ", len(all_combs)* len(age_query_strings) * len(balance_query_strings))

            # We can now go ahead and genreate the feature sets based on what was done previously.

            num_iter = 0
            combs_to_consider = {}
            # Setting up looping structures to generate all possibilities.
            # All that has to be done now is to change 'df_train' to 'X_train'.
            for age_query in age_query_strings:
                df_filtered_final = train_df.query(age_query)
                for bal_query in balance_query_strings:
                    df_filtered_final = df_filtered_final.query(bal_query)
                    for comb in all_combs:
                        dict_final_query = construct_dict(comb)
                        num_iter += 1
                        extracted_df = extract_rows_feature_set(df_filtered_final, dict_final_query)
                        key = (dict_final_query['education'], dict_final_query['job'], dict_final_query['marital'], dict_final_query['default'], dict_final_query['loan'], dict_final_query['housing'], bal_query, age_query)
                        n_rows = extracted_df.shape[0]
                        if n_rows >0:
                            results = compute_expected_succ_per_call_rate_feature_set(extracted_df, max_calls)
                            max_loc = compute_optimal_call_no(results)
                            rate = results[max_loc]['expected']
                            # In this new case max_loc never goes below zero!
                            if max_loc != -1:
                                combs_to_consider[key] = {'age':age_query, 'bal':bal_query, 'comb':comb, 'consider':True, 'max_loc':max_loc, 'rate':results[max_loc]['expected'], 'n_rows':n_rows}
#                             print(comb)
#                             print("Max Loc is: ", max_loc+1)
#                             plot_graph_new(results, max_calls, False, "Expected Ratio per Call")
        #  When we are finished creating the feature combinations .... we can now use the hold out set for validation of the model!
            # Baseline Metrics
            num_succ = 0
            num_calls = 0
            # Optimal Method Metrics
            num_succ_optimal = 0
            num_calls_optimal = 0
            num_bad_cons = 0
            num_good_cons = 0

            num_fc_small_sample_size = 0

            all_possible_calls = []

            for loc, row in test_df.iterrows():
                # For optimal method.
                # We have the exact values for each of the following:
                jb_query = convert(find_matching_attribute_comb(str(row['job']), job_cmbs))
                mt_query = convert(find_matching_attribute_comb(str(row['marital']), marital_cmbs))
                ec_query = convert(find_matching_attribute_comb(str(row['education']), education_cmbs))
                house_query = convert(find_matching_attribute_comb(str(row['housing']), housing_cmbs))
                loan_query = convert(find_matching_attribute_comb(str(row['loan']), loan_cmbs))
                def_query = convert(find_matching_attribute_comb(str(row['default']), default_cmbs))
                ##########################
                no_calls = row['campaign']
                # The balance and age are within ranges so we need to find the matching query.
                ##########################
                balance = row['balance']
                bal_query = None
                age = row['age']
                age_query = None
                for age_q in age_query_strings:
                    if eval(age_q):
                        age_query = age_q
                for bal_q in balance_query_strings:
                    if eval(bal_q):
                        bal_query = bal_q
                key_to_find = (ec_query, jb_query, mt_query, def_query, loan_query, house_query, bal_query, age_query)
                if key_to_find in combs_to_consider.keys():
                    fs = combs_to_consider[key_to_find]
                    # Adding the rate, feature set, the outcome and the number of calls made to a new list .. which will be sorted afterwards.
                    if fs['n_rows'] >= 20:
                        all_possible_calls.append((fs['rate'], fs, row['y'], row['campaign']))
                    else:
                        num_fc_small_sample_size += 1

            # Actual Testing ..

            total_possible_calls = call_limit
            all_possible_calls_sorted = sorted(all_possible_calls, key = lambda tup: tup[0], reverse = True)
            subset_df = test_df.sample(n = total_possible_calls)

            # Baseline 
            for loc, row in subset_df.iterrows():
                if num_calls + row['campaign'] <= total_possible_calls:
                    num_calls += row['campaign']
                    if row['y'] == "yes":
                        num_succ += 1
            ratio_baseline = div(num_succ, num_calls)
            all_non_optimal_ratios.append(ratio_baseline)
#             print("Ratio for baseline: ", ratio_baseline)

            # Optimal
            for item in all_possible_calls_sorted:
                if item[3] + num_calls_optimal <= total_possible_calls:
                    num_calls_optimal += item[3]
                    if item[2] == "yes":
                        num_succ_optimal += 1
            ratio_optimal = div(num_succ_optimal, num_calls_optimal)
            all_optimal_ratios.append(ratio_optimal)
#             print("Ratio for optimal: ", ratio_optimal)  
#             print("Number of FS with small sample sizes: ", num_fc_small_sample_size)
#             print("\n")
    mean_non_optimal = statistics.mean(all_non_optimal_ratios)
    mean_optimal = statistics.mean(all_optimal_ratios)
    std_dev_non_optimal = statistics.stdev(all_non_optimal_ratios)
    std_dev_optimal = statistics.stdev(all_optimal_ratios)
    
#     print("Mean Non-Optimal: ", mean_non_optimal)
#     print("Mean Optimal: ", mean_optimal)
#     print("Std Dev Non-Optimal: ", std_dev_non_optimal)
#     print("Std Dev Optimal: ", std_dev_optimal)
#     print(all_non_optimal_ratios)
#     print(all_optimal_ratios)
    
    f = open(str(call_limit)+".txt","w+")
    f.write("%s\n" % str(mean_non_optimal))
    f.write("%s\n" % str(mean_optimal))
    f.write("%s\n" % str(std_dev_non_optimal))
    f.write("%s\n" % str(std_dev_optimal))
    f.close()

KeyboardInterrupt: 

In [ ]:
age = 45
for age_q in age_query_strings:
    if eval(age_q):
        print("Yes!")
        print(age_q)

## Age and balance computation.

In [32]:
# Age.
# all_age_query_strings = ['age >= 10 & age <= 32', 'age >= 33 & age <= 40', 'age >= 50 & age <= 59', 'age >= 60']
# all_age_query_strings = ['age >= 10 & age <= 34', 'age >= 35 & age <= 45', 'age >= 46']
all_age_query_strings = ['age < 26','age >= 26 & age <=60','age >60']
for age_query in all_age_query_strings:
    df_filtered_final = mkt_df_filtered.query(age_query)
    print(df_filtered_final.shape)
    print(age_query, compute_metric(df_filtered_final))

(1326, 17)
age < 26 0.11134307585247043
(42453, 17)
age >= 26 & age <=60 0.03950606355669647
(1188, 17)
age >60 0.2084717607973422


In [34]:
# Age.
all_age_query_strings = ['age >= 10 & age <= 19', 'age >= 20 & age <= 29', 'age >= 30 & age <= 39', 'age >= 40 & age <= 49', 'age >= 50 & age <= 59','age >= 60 & age <= 69', 'age >= 70 & age <= 79', 'age >= 80 & age <= 100']
# all_age_query_strings = ['age >= 10 & age <= 100']
for age_query in all_age_query_strings:
    df_filtered_final = mkt_df_filtered.query(age_query)
    print(df_filtered_final.shape)
    print(age_query, len(df_filtered_final), compute_metric(df_filtered_final))

(47, 17)
age >= 10 & age <= 19 47 0.14634146341463414
(5189, 17)
age >= 20 & age <= 29 5189 0.07631601041054488
(17989, 17)
age >= 30 & age <= 39 17989 0.040097307272879794
(11584, 17)
age >= 40 & age <= 49 11584 0.03357253501090633
(8375, 17)
age >= 50 & age <= 59 8375 0.03437390389337075
(1229, 17)
age >= 60 & age <= 69 1229 0.12427647259107934
(424, 17)
age >= 70 & age <= 79 424 0.20594965675057209
(130, 17)
age >= 80 & age <= 100 130 0.1950354609929078


In [ ]:
# Age
all_age_query_tuples = [(10, 20), (20, 30), (30, 40), (50, 60), (60, 70), (70, 80), (80, 90), (90, 100)]
ratios, all_age_query_strings = compute_metric_for_each_attribute_range(all_age_query_tuples, train_df, 'age')
find_combinations(all_age_query_strings, ratios)

## ---------------------------------------------------------------------------------

In [ ]:
# Balance
all_bal_query_strings = ['balance >= -100000 & balance <= -1', 'balance >= 0 & balance < 250', 'balance >= 250 & balance < 500','balance >= 500 & balance < 750', 'balance >= 750 & balance < 1000', 'balance >= 1000 & balance < 2000', 'balance >= 2000 & balance < 3000', 'balance >= 3000 & balance < 4000','balance >= 4000 & balance < 5000', 'balance >= 5000 & balance < 6000', 'balance >= 6000 & balance < 7000', 'balance >= 7000 & balance < 8000', 'balance >= 8000 & balance < 9000', 'balance >= 9000 & balance < 10000','balance >= 10000 & balance < 11000', 'balance >= 11000 & balance < 12000', 'balance >= 12000 & balance < 13000', 'balance >= 13000 & balance < 14000', 'balance >= 14000 & balance < 15000', 'balance >= 15000 & balance < 16000', 'balance >= 16000 & balance < 17000','balance >= 17000 & balance < 18000', 'balance >= 18000 & balance < 19000', 'balance >= 19000 & balance < 19000', 'balance >= 20000']
for bal_query in all_bal_query_strings:
    df_filtered_final = mkt_df_filtered.query(bal_query)
    print(bal_query, len(df_filtered_final), compute_metric(df_filtered_final))

In [ ]:
# Balance
all_bal_query_tuples = [(-10000, 0), (0, 250), (250, 500), (500, 750), (750,1000), (1000, 2000), (2000, 3000), (3000, 4000), (4000, 5000), (5000, 6000), (6000, 7000), (8000, 100000)]
ratios, all_bal_query_strings = compute_metric_for_each_attribute_range(all_bal_query_tuples, train_df, 'balance')
find_combinations(all_bal_query_strings, ratios)

## Determining where to stop regarding the number of calls.

In [ ]:
# This is to determine the maximum number of calls we should stop at!
all_ratios_calls = []
for i in range(1,57):
    query_str = 'campaign == ' + str(i)
    call_query_data = mkt_df_filtered.query(query_str)
    succ = 0
    calls = 0
    for lc, rw in call_query_data.iterrows():
        if rw['y'] == "yes":
            succ += 1
        calls += rw['campaign']
    all_ratios_calls.append(div(succ, calls))
for index, value in enumerate(all_ratios_calls):
    print(index+1, value)
plot_graph_new(all_ratios_calls, 56, True)

## Improving the success rate by optimizing the maximum calls made.

In [ ]:
mkt_df_filtered.head()

In [ ]:
max_calls_considered = 20

current_dir = os.getcwd()
mkt_df = load_file(current_dir + '/bank-full.csv')
mkt_df_filtered = mkt_df[(mkt_df['campaign']>=1) & (mkt_df['campaign']<=max_calls_considered)]

result_ratios = [0.0 for i in range (1,max_calls_considered+1)]

for i in range(1, max_calls_considered+1):
    total_calls = 0
    total_succ = 0
    #query_str = "campaign <= {0}".format(i)
    #print(query_str)
    #df_filtered_campaign = mkt_df_filtered.query(query_str)
    for loc, row in mkt_df_filtered.iterrows():
        if row['y']  == "yes" and row['campaign'] <= i:
            total_succ += 1
        total_calls += min(i, row['campaign'])
    result_ratios[i-1] = div(total_succ , total_calls)
    print(i, result_ratios[i-1], total_succ, total_calls)

In [ ]:
plot_graph_new(result_ratios, 20, True, "Ratio Per Call #")

In [ ]:
current_dir = os.getcwd()
mkt_df = load_file(current_dir + '/bank-full.csv')
mkt_df_filtered = mkt_df[(mkt_df['campaign']>=1) & (mkt_df['campaign']<=max_calls)]
mkt_df_filtered.shape

In [ ]:
mkt_df_filtered.head(n=10)

In [ ]:
mkt_df_filtered['poutcome'].value_counts()

In [ ]:
mkt_df_filtered_successes = mkt_df_filtered.query("poutcome == 'success'")
print(mkt_df_filtered_successes.shape)
mkt_df_filtered_successes['previous'].value_counts()

In [ ]:
mkt_df_filtered_successes['poutcome'].value_counts()

In [ ]:
res = mkt_df_filtered_successes['campaign'].value_counts(normalize = False)
print(res)
print(res.values)
num_succ = [2561, 1401 , 618, 317, 139, 92, 47, 32, 21, 14, 16, 4, 6, 4, 4, 2, 6, 0, 0 ,1]

In [ ]:
plot_graph_new(num_succ, 20, True, "Frequency of Contacts Made per Call #")

In [ ]:
a = mkt_df_filtered.query("y == 'yes'").shape[0]
b = mkt_df_filtered.query("poutcome == 'success'").shape[0]
anb = mkt_df_filtered.query("y == 'yes' and poutcome == 'success'").shape[0]
print(anb/b)

In [ ]:
for i in range(0, 4920, 60):
    res = mkt_df_filtered.query("duration >= {0} and duration <= {1}".format(str(i-60), str(i)))
    print(i, res.shape[0], res.query("y == 'yes'").shape[0])
# a = mkt_df_filtered.query("duration >= 0 and duration <= 180").shape[0]
# b = mkt_df_filtered.query("y == 'yes'").shape[0]
# anb = mkt_df_filtered.query("y == 'yes' and duration >= 0 and duration <= 1000").shape[0]
# print(anb/b)

In [ ]:
# int(mkt_df_filtered['duration'].max()/60)+1
int(mkt_df_filtered['duration'].max()/60) + 1

In [ ]:
a = mkt_df_filtered.query("y == 'yes' and contact == 'cellular'").shape[0]
b = mkt_df_filtered.query("y == 'yes' and contact == 'telephone'").shape[0]
c = mkt_df_filtered.query('y == "yes"').shape[0]
print(a, b, c)